In [46]:
# doc = nlp(SOME_LONG_TEXT_HERE)
# ents = [ent for ent in list(doc.ents) if ent.label_ != 'DATE']

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import string
import nltk
from nltk.stem import SnowballStemmer
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

from sklearn.metrics import accuracy_score


from subprocess import check_output

In [47]:
# Import the email modules we'll need
import glob
import email
import mailparser
from email import policy
from email.parser import BytesParser

path = '../datawe/raw/Email_Classification/*'
email_types = glob.glob(path)
appendFilesData = []
file_raw_data = [] 
for folder in email_types:
    files = glob.glob(folder+"/*.txt")
    email_type = folder.split('\\')[1]
    for name in files:
        try:
            with open(name) as fp:
                raw_data = fp.read()
                file_raw_data.append(raw_data)
                msg = mailparser.parse_from_string(raw_data)
                appendFilesData.append({
                    "to":msg.to,
                    "from":msg.from_,
                    "subject":msg.subject,
                    "date":msg.date,
#                     "sent":msg["Sent"],
#                     "importance":msg["Importance"],
                    "content": raw_data, #msg.body,
                    "class_to_exec":email_type,
                })
         
        except IOError as exc:
            print('Exception')

            
#creating pandas dataframe
data = pd.DataFrame(appendFilesData)

In [48]:
##text cleaning
def clean(doc):
    words_to_exclude = set(stopwords.words('english'))
    exclude = set(string.punctuation)
    
#     doc = doc.replace('\n', ' ')
    tagged_sentence = nltk.tag.pos_tag(doc.split())
    edited_sentence = ' '.join([word for word,tag in tagged_sentence if tag != 'NNP' and tag != 'NNPS'])

    word_free = " ".join([i for i in edited_sentence.lower().split() if i not in words_to_exclude])
    punc_free = ''.join(ch for ch in word_free if ch not in exclude)
    alpha_free = " ".join(word for word in punc_free.split() if word.isalpha())
    stemmer = SnowballStemmer("english").stem
    stem_free = " ".join(stemmer(stem) for stem in alpha_free.split())
    lemma = WordNetLemmatizer()
    normalized = " ".join(lemma.lemmatize(word) for word in stem_free.split())
    
#     print(edited_sentence)
#     print(word_free)
#     print(punc_free)
#     print(alpha_free)
#     print(stem_free)
    

    return normalized



In [49]:
## text transform 
textFeatures = data['content'].copy()
textFeatures = textFeatures.apply(clean)
vectorizer =  TfidfVectorizer()#TfidfVectorizer("english") 
features = vectorizer.fit_transform(textFeatures)

features_train, features_test, labels_train, labels_test = train_test_split(features, data['class_to_exec'], test_size=0.2, random_state=111)

In [50]:
from sklearn.naive_bayes import MultinomialNB

mnb = MultinomialNB(alpha=0.2)
mnb.fit(features_train, labels_train)
prediction = mnb.predict(features_test)
accuracy_score(labels_test,prediction)

0.6666666666666666

In [51]:
## grid serach CV for tuning hyperparameters
import sklearn.model_selection as modsel
from sklearn.linear_model import LogisticRegression
param_grid_ = {'C': [1e-5, 1e-3, 1e-1, 1e0, 1e1, 1e2]}

tfidf_search = modsel.GridSearchCV(LogisticRegression(), cv=5, param_grid=param_grid_)
tfidf_search.fit(features_train, labels_train)

c:\users\lokesh\appdata\local\programs\python\python36\lib\site-packages\sklearn\model_selection\_split.py:626: Warning: The least populated class in y has only 4 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)
c:\users\lokesh\appdata\local\programs\python\python36\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
c:\users\lokesh\appdata\local\programs\python\python36\lib\site-packages\sklearn\linear_model\logistic.py:459: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
c:\users\lokesh\appdata\local\programs\python\python36\lib\site-packages\sklearn\model_selection\_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from 

GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'C': [1e-05, 0.001, 0.1, 1.0, 10.0, 100.0]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [52]:
# vectorizer.vocabulary_

pre = tfidf_search.predict(features_test)
accuracy_score(labels_test,pre)

0.8333333333333334